In [3]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(""), '..', 'Lime_NDT')))
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(""), '..')))

In [4]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from Lime_NDT.lime_tabular import LimeTabularExplainer
from Lime_NDT.utils.ndt_sklearn_wrapper import NDTRegressorWrapper

# Charger le dataset Iris
iris = load_iris()
X = iris.data
y = iris.target  # Pour la régression, on garde les labels numériques

# Séparer en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Entraîner le modèle global Random Forest
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)

# Fonction de prédiction globale utilisant le Random Forest
def predict_fn(X):
    return rf.predict(X)

# Créer l'explainer LIME
explainer = LimeTabularExplainer(
    X_train,
    feature_names=iris.feature_names,
    class_names=iris.target_names,
    discretize_continuous=True,
    mode='regression'
)

# Créer le modèle local NDT
model_regressor = NDTRegressorWrapper(D=X_train.shape[1])

# Expliquer une instance
exp = explainer.explain_instance(
    X_test[0],
    predict_fn,
    num_features=4,
    model_regressor=model_regressor
)

print("Explication locale (feature, poids):")
print(exp.as_list())

mean_leaf_values shape: (16, 1, 1)
self.L: 16 self.C: 1
mean_leaf_values shape after squeeze: (16, 1)


c:\Users\DELL\Desktop\Lime_NDT\.venv\lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Explication locale (feature, poids):
[('5.80 < sepal length (cm) <= 6.40', 0.4666666666666667), ('sepal width (cm) <= 2.80', 0.3333333333333333), ('0.30 < petal width (cm) <= 1.30', 0.13333333333333333), ('4.30 < petal length (cm) <= 5.10', 0.06666666666666667)]


In [5]:
from sklearn.linear_model import Ridge

# Créer le modèle local Ridge (régression linéaire)
ridge_regressor = Ridge(alpha=1.0, fit_intercept=True, random_state=42)

# Expliquer la même instance avec Ridge comme modèle local
exp_ridge = explainer.explain_instance(
    X_test[0],
    predict_fn,
    num_features=4,
    model_regressor=ridge_regressor
)

print("\nExplication locale avec Ridge (feature, poids):")
print(exp_ridge.as_list())


Explication locale avec Ridge (feature, poids):
[('4.30 < petal length (cm) <= 5.10', 0.1523554231485901), ('0.30 < petal width (cm) <= 1.30', -0.07196461596433029), ('sepal width (cm) <= 2.80', 0.04622829503013914), ('5.80 < sepal length (cm) <= 6.40', -0.009164312063786902)]
